<a href="https://colab.research.google.com/github/malloyca/CSC581B/blob/main/Final%20Project/small_convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose # todo: Lambda, Compose not necessary?
import matplotlib.pyplot as plt
import copy

In [2]:
# Load the training data (CIFAR10 to start)
training_data = datasets.CIFAR10(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

# Load the test data
test_data = datasets.CIFAR10(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
# Check the data dimensions
for X, y in test_dataloader:
  print("Shape of X batch [Batch size, Channels, Height, Width]: ", X.shape)
  print("Shape of y batch: ", y.shape, y.dtype)
  break

Shape of X batch [Batch size, Channels, Height, Width]:  torch.Size([64, 3, 32, 32])
Shape of y batch:  torch.Size([64]) torch.int64


# Building basic convolutional neural nets

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
# Define the model
class basicConvNet(nn.Module):
  def __init__(self, name):
    super(basicConvNet, self).__init__()
    self.name = name
    self.basic_conv_net = nn.Sequential(
        # 1st layer
        nn.Conv2d(3, 64, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d((2,2)),

        # 2nd layer
        nn.Conv2d(64, 32, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d((2,2)),

        # 3rd layer
        nn.Conv2d(32, 32, (3,3), padding='same'),
        nn.ReLU(),
        nn.MaxPool2d(2,2),

        # Last layer
        nn.Flatten(),
        nn.Linear(32 * 4 * 4, 10),
    )
    self.softmax = nn.Softmax(dim=1)

  # Forward propagation
  def forward(self, x):
    x = self.basic_conv_net(x)
    x = self.softmax(x)
    return x
      

In [7]:
model = basicConvNet('test').to(device)

In [8]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [9]:
# Training loop
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [10]:
# Test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return test_loss

In [11]:
n_epochs = 5

In [12]:
def training_loop(n_epochs, model, train_data, test_data, loss_function,
                  optimizer):
  
  for e in range(n_epochs):
    print(f"Epoch {e+1}\n------------------------")
    train(train_data, model, loss_function, optimizer)
    test_loss = test(test_data, model, loss_function)

In [13]:
training_loop(n_epochs, model, train_dataloader, test_dataloader, loss_fn, optimizer)

Epoch 1
------------------------
loss: 2.301640 [    0/50000]


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss: 2.302561 [ 6400/50000]
loss: 2.302672 [12800/50000]
loss: 2.301651 [19200/50000]
loss: 2.302730 [25600/50000]
loss: 2.302497 [32000/50000]
loss: 2.302310 [38400/50000]
loss: 2.302088 [44800/50000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302449 

Epoch 2
------------------------
loss: 2.301622 [    0/50000]
loss: 2.302496 [ 6400/50000]
loss: 2.302319 [12800/50000]
loss: 2.301740 [19200/50000]
loss: 2.302243 [25600/50000]
loss: 2.302267 [32000/50000]
loss: 2.302129 [38400/50000]
loss: 2.302005 [44800/50000]
Test Error: 
 Accuracy: 10.1%, Avg loss: 2.302175 

Epoch 3
------------------------
loss: 2.301563 [    0/50000]
loss: 2.302473 [ 6400/50000]
loss: 2.301825 [12800/50000]
loss: 2.301808 [19200/50000]
loss: 2.301620 [25600/50000]
loss: 2.301861 [32000/50000]
loss: 2.301940 [38400/50000]
loss: 2.301806 [44800/50000]
Test Error: 
 Accuracy: 13.1%, Avg loss: 2.301725 

Epoch 4
------------------------
loss: 2.301477 [    0/50000]
loss: 2.302483 [ 6400/50000]
loss: 2.300823 [1280